### Set Path (Won't be needed once `setup.py` is finished)

In [1]:
import sys
sys.path.insert(0, sys.path[0][:-8])

In [2]:
import torch
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.metrics import mean_absolute_error

### Auglichem imports

In [3]:
from auglichem.crystal import Compose, RotationTransformation, SupercellTransformation
from auglichem.crystal.data import CrystalDatasetWrapper
from auglichem.crystal.models import CrystalGraphConvNet as CGCNN
from auglichem.crystal.models import SchNet
from auglichem.crystal.models import GINet

### Set up dataset

In [4]:
#help(CrystalDatasetWrapper)

In [5]:
#help(CrystalDatasetWrapper.__init__)

In [8]:
# Create transformation
transform = [
    SupercellTransformation(),
    RotationTransformation()
]

# Initialize dataset object
dataset = CrystalDatasetWrapper("lanthanides", batch_size=256,
                                valid_size=0.1, test_size=0.1)

# Get train/valid/test splits as loaders
train_loader, valid_loader, test_loader = dataset.get_data_loaders(transform=transform)

100%|█████████████████████████████████████████████████████████| 3332/3332 [00:00<00:00, 34920.48it/s]


### Initialize model with task from data

In [9]:
# Get model
structures, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

model = CGCNN(orig_atom_fea_len, nbr_fea_len)

/Users/clo/miniforge3/envs/auglichem_3.8/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


### Initialize traning loop

In [10]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

### Train the model

In [11]:
for epoch in range(1):
    for bn, (data, target, _) in tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        input_var = (Variable(data[0]),
                     Variable(data[1]),
                     data[2],
                     data[3])
        
        pred = model(*input_var)
        loss = criterion(pred, target)
        
        loss.backward()
        optimizer.step()

39it [03:16,  5.04s/it]


### Test the model

In [ ]:
def evaluate(model, test_loader):
    with torch.no_grad():
        model.eval()
        data, target, _ = next(iter(test_loader))
        input_var = (Variable(data[0]),
                     Variable(data[1]),
                     data[2],
                     data[3])

        pred = model(*input_var)
        mae = mean_absolute_error(pred, target)   
        
    print("TEST MAE: {0:.3f}".format(mae))

In [13]:
evaluate(model, test_loader)

TypeError: cannot unpack non-iterable int object

### Model saving/loading example

In [ ]:
# Save model
torch.save(model.state_dict(), "./saved_models/example_cgcnn")

In [ ]:
# Instantiate new model and evaluate
structures, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

model = CGCNN(orig_atom_fea_len, nbr_fea_len)

evaluate(model, test_loader)

In [ ]:
# Load saved model and evaluate
model.load_state_dict(torch.load("./saved_models/example_cgcnn"))
evaluate(model, test_loader)